Implementing self-editing memory from scratch

### Section 0: Setup OpenAI

In [2]:
import json
import random
from helper import set_openai_key, test_openai_api, create_openai_client, print_pretty

set_openai_key()

test_openai_api()

client = create_openai_client()

API key set successfully.
Hello! How can I assist you today?


### Section 1: Breaking down the LLM context window

#### A simple agent's context window

In [5]:
model = "gpt-4o-mini"

system_prompt = "You are a chatbot."

# Make the completion request with the tool usage
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt: always included in the context window 
        {"role": "system", "content": system_prompt}, 
        # chat history (evolves over time)
        {"role": "user", "content": "What is my name?"}, 
    ]
)
chat_completion.choices[0].message.content

"I'm sorry, but I don't have access to personal information, including your name. How can I assist you today?"

#### Adding memory to the context

In [6]:
agent_memory = {"human": "Name: Bob"}

system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

In [7]:
import json

chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt + "[MEMORY]\n" + \
         json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "What is my name?"},
    ],
)
chat_completion.choices[0].message.content

'Your name is Bob.'

### Section 2: Modifing the memory with tools

#### Defining a memory editing tool

In [8]:
agent_memory = {"human": "", "agent": ""}

def core_memory_save(section: str, memory: str):
    # Append the memory of specific section
    agent_memory[section] += '\n' 
    agent_memory[section] += memory 

In [9]:
agent_memory

{'human': '', 'agent': ''}

In [10]:
core_memory_save("human", "The human's name is Charles")

In [11]:
agent_memory

{'human': "\nThe human's name is Charles", 'agent': ''}

Let's define a tool schema for the `core_memory_save` function that GPT can call.

In [12]:
# tool description 
core_memory_save_description = "Save important information about you," \
+ "the agent or the human you are chatting with."

core_memory_save_description

'Save important information about you,the agent or the human you are chatting with.'

In [13]:
# arguments into the tool (generated by the LLM)
# defines what the agent must generate to input into the tool 
core_memory_save_properties = \
{
    # arg 1: section of memory to edit
    "section": {
        "type": "string",
        "enum": ["human", "agent"],
        "description": "Must be either 'human' " \
        + "(to save information about the human) or 'agent'" \
        + "(to save information about yourself)",            
    },
    # arg 2: memory to save
    "memory": {
        "type": "string",
        "description": "Memory to save in the section",
    },
}

In [14]:
# tool schema (passed to OpenAI)
core_memory_save_metadata = \
    {
        "type": "function",
        "function": {
            "name": "core_memory_save",
            "description": core_memory_save_description,
            "parameters": {
                "type": "object",
                "properties": core_memory_save_properties,
                "required": ["section", "memory"],
            },
        }
    }

#### Run first agent step

In [15]:
agent_memory = {"human": ""}
system_prompt = "You are a chatbot. " \
+ "You have a section of your context called [MEMORY] " \
+ "that contains information relevant to your conversation"

chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "My name is Bob"},
    ],
    # tool schemas 
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_ltdzf4L1kzuIkVjUvtNOXrQK', function=Function(arguments='{"section":"human","memory":"Name is Bob"}', name='core_memory_save'), type='function')]))

In [17]:
print_pretty(response.dict())

{
    "finish_reason": "tool_calls",
    "index": 0,
    "logprobs": null,
    "message": {
        "audio": null,
        "content": null,
        "function_call": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
            {
                "function": {
                    "arguments": "{\"section\":\"human\",\"memory\":\"Name is Bob\"}",
                    "name": "core_memory_save"
                },
                "id": "call_ltdzf4L1kzuIkVjUvtNOXrQK",
                "type": "function"
            }
        ]
    }
}


#### Executing the tool

In [18]:
arguments = json.loads(response.message.tool_calls[0].function.arguments)
arguments

{'section': 'human', 'memory': 'Name is Bob'}

In [20]:
# run the function with the specified arguments 
core_memory_save(**arguments)

In [21]:
agent_memory


{'human': '\nName is Bob'}

#### Running the next agent step

In [22]:
chat_completion = client.chat.completions.create(
    model=model,
    messages=[
        # system prompt 
        {"role": "system", "content": system_prompt}, 
        # memory 
        {"role": "system", "content": "[MEMORY]\n" + json.dumps(agent_memory)},
        # chat history 
        {"role": "user", "content": "what is my name"},
    ],
    tools=[core_memory_save_metadata]
)
response = chat_completion.choices[0]
response.message


ChatCompletionMessage(content='Your name is Bob.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

#### Implementing an agentic loop

In [24]:
# Init agent memory
agent_memory = {"human": ""}

In [30]:
system_prompt_os = system_prompt \
+ "\n. You must either call a tool (core_memory_save) or" \
+ "write a response to the user. " \
+ "Do not take the same actions multiple times!" \
+ "When you learn new information, make sure to always" \
+ "call the core_memory_save tool." 

In [31]:
def agent_step(user_message): 
    
    # prefix messages with system prompt and memory
    messages = [
        # system prompt 
        {"role": "system", "content": system_prompt_os}, 
        # memory
        {
            "role": "system", 
            "content": "[MEMORY]\n" + json.dumps(agent_memory)
        },
    ]    

    # append the most recent message
    messages.append({"role": "user", "content": user_message})
    
    # agentic loop 
    while True:
        chat_completion = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=[core_memory_save_metadata]
        )
        response = chat_completion.choices[0]

        # update the messages with the agent's response 
        messages.append(response.message)
        
        # if NOT calling a tool (responding to the user), return 
        if not response.message.tool_calls: 
            return response.message.content

        # if calling a tool, execute the tool
        else: 
            print("TOOL CALL:", response.message.tool_calls[0].function)
            
            # parse the arguments from the LLM function call
            arguments = json.loads(
                response.message.tool_calls[0].function.arguments
            )

            # run the function with the specified arguments
            core_memory_save(**arguments)

            # add the tool call response to the message history 
            messages.append({
                "role": "tool", 
                "tool_call_id": response.message.tool_calls[0].id, 
                "name": "core_memory_save", 
                "content": f"Updated memory: {json.dumps(agent_memory)}"
            })

In [27]:
agent_step("my name is bob.")

TOOL CALL: Function(arguments='{"section":"human","memory":"The user\'s name is Bob."}', name='core_memory_save')


'Nice to meet you, Bob! How can I assist you today?'

In [29]:
agent_step("What is my name?")

'Your name is Bob.'